# Sentiment Analysis on IMDB dataset

This notebook walks through how perform sentament analysis on the IMDB dataset.
In this setting, one party has the reviews and the other party has the labels.
The party with the labels is helping the party with the reviews train a model
without sharing the labels themselves.

Before starting, install tf-shell and the dataset.

```bash
pip install tf-shell
pip install tensorflow_hub tensorflow_datasets
```

In [1]:
import time
from datetime import datetime
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import keras
import numpy as np
import tf_shell
import tf_shell_ml
import os

2024-08-21 15:43:44.632717: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-21 15:43:44.655330: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

# full_range = 33
# num_threads = 3

# a = list(range(0, 10))
# b = list(range(10, 20))
# c = list(range(20, 33))

# # import math
# # r = math.ceil(33 / 13)
# # print(r)

# aa = [i * 11 for i in a]
# bb = [i * 11 for i in b]
# cc = [i * 11 for i in c]

# aa = [aaa%33 for aaa in aa]
# bb = [bbb%33 for bbb in bb]
# cc = [ccc%33 for ccc in cc]

# print(aa)
# print(bb)
# print(cc)

# print(sorted(aa + bb + cc))

In [3]:
# Set up parameters for the SHELL encryption library.
context = tf_shell.create_context64(
    log_n=12,
    main_moduli=[288230376151760897, 288230376152137729],
    plaintext_modulus=4294991873,
    scaling_factor=3,
    mul_depth_supported=3,
    seed="test_seed",
)

# Create the secret key for encryption and a rotation key (rotation key is
# an auxilary key required for operations like roll or matmul).
secret_key = tf_shell.create_key64(context)
public_rotation_key = tf_shell.create_rotation_key64(context, secret_key)

# The batch size is determined by the ciphertext parameters, specifically the
# schemes polynomial's ring degree because tf-shell uses batch axis packing.
# Furthermore, two micro-batches to run in parallel.
batch_size = context.num_slots

use_encryption = True

Setup IMDB dataset.

In [4]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, val_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

# Print the first example.
for review, label in train_data.take(1):
    print("Review:", review.numpy().decode('utf-8'))
    print("Label:", label.numpy())

epochs = 10
train_data = train_data.shuffle(buffer_size=2048).batch(batch_size, drop_remainder=True).repeat(count=epochs)
val_data = val_data.shuffle(buffer_size=2048).batch(batch_size, drop_remainder=True)
test_data = test_data.shuffle(buffer_size=2048).batch(batch_size, drop_remainder=True)

vocab_size = 10000  # This dataset has 92061 unique words.
max_length = 250
embedding_dim = 16

vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
)

vectorize_layer.adapt(train_data.map(lambda text, label: text))

print("Most used words:", vectorize_layer.get_vocabulary()[:10])
print("Dictionary size:", len(vectorize_layer.get_vocabulary()))

# Count the top n words in the training set.
top_n = 200
word_counts = np.zeros(top_n, dtype=np.int64)
for review, label in train_data:
    vectorized_reviews = vectorize_layer(review)
    for i in range(len(word_counts)):
        counts = tf.where(vectorized_reviews == i, 1, 0)
        word_counts[i] += tf.reduce_sum(tf.cast(counts, dtype=tf.int64))

for i in range(len(word_counts)):
    print(f"Word {i} ({vectorize_layer.get_vocabulary()[i]}) count: {word_counts[i]}")

2024-08-21 15:43:59.362596: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-08-21 15:43:59.363258: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
Label: 0


2024-08-21 15:44:03.915397: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Most used words: ['', '[UNK]', 'the', 'a', 'and', 'of', 'to', 'is', 'in', 'it']
Dictionary size: 10000


2024-08-21 15:45:02.520708: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Word 0 () count: 193376173
Word 1 ([UNK]) count: 1909588
Word 2 (the) count: 1640160
Word 3 (a) count: 793567
Word 4 (and) count: 792777
Word 5 (of) count: 713473
Word 6 (to) count: 663458
Word 7 (is) count: 521799
Word 8 (in) count: 454077
Word 9 (it) count: 378937
Word 10 (i) count: 376537
Word 11 (this) count: 372522
Word 12 (that) count: 342774
Word 13 (br) count: 281142
Word 14 (was) count: 238349
Word 15 (as) count: 226955
Word 16 (for) count: 214889
Word 17 (with) count: 215272
Word 18 (movie) count: 205534
Word 19 (but) count: 204017
Word 20 (film) count: 184764
Word 21 (on) count: 165747
Word 22 (not) count: 147455
Word 23 (you) count: 146662
Word 24 (are) count: 144779
Word 25 (his) count: 143958
Word 26 (have) count: 137439
Word 27 (be) count: 130597
Word 28 (he) count: 129849
Word 29 (one) count: 125283
Word 30 (its) count: 122528
Word 31 (all) count: 115207
Word 32 (at) count: 114532
Word 33 (by) count: 109142
Word 34 (they) count: 104436
Word 35 (an) count: 104161
Word 36

In [5]:
# Create the trainable layers.
embedding_layer = tf_shell_ml.ShellEmbedding(
    vocab_size + 1,  # +1 for OOV token.
    embedding_dim,
    skip_embeddings_below_index=top_n,
)
# TODO dropout layer?
hidden_layer = tf_shell_ml.GlobalAveragePooling1D()
# TODO dropout layer?
output_layer = tf_shell_ml.ShellDense(
    2,
    activation=tf.nn.softmax,
)

layers = [
    embedding_layer,
    hidden_layer,
    output_layer,
]

loss_fn = tf_shell_ml.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(0.1)

Next, define the `train_step` function which will be called for each batch on an
encrypted batch of labels, y. The function first does a forward on the plaintext
image x to compute a predicted label, then does backpropagation using the
encrypted label y.

In [6]:
def train_step(x, enc_y):
    # Forward pass always in plaintext
    y_pred = x
    for i, l in enumerate(layers):
        y_pred = l(y_pred)

    # Backward pass.
    dx = loss_fn.grad(enc_y, y_pred)
    dJ_dw = []
    dJ_dx = [dx,]
    for l in reversed(layers):
        if isinstance(l, tf_shell_ml.GlobalAveragePooling1D):
            dw, dx = l.backward(dJ_dx[-1])
        else:
            dw, dx = l.backward(dJ_dx[-1], public_rotation_key)
        dJ_dw.extend(dw)
        dJ_dx.append(dx)

    return reversed(dJ_dw)


@tf.function
def train_step_wrapper(x_batch, y_batch):
    if use_encryption:
        # Encrypt the batch of secret labels y.
        enc_y_batch = tf_shell.to_encrypted(y_batch, secret_key, context)
    else:
        enc_y_batch = y_batch

    # Run the training step. The top and bottom halves of the batch are
    # treated as two separate mini-batches run in parallel.
    enc_grads = train_step(x_batch, enc_y_batch)

    filtered_layers = [l for l in layers if len(l.weights) > 0]

    if use_encryption:
        # Decrypt the weight gradients. In practice, the gradients should be
        # noised before decrypting.
        packed_grads = [tf_shell.to_tensorflow(g, secret_key) for g in enc_grads]
        # Unpack the plaintext gradients using the corresponding layer.
        grads = [l.unpack(g) for l, g in zip (filtered_layers, packed_grads)]
    else:
        grads = enc_grads

    weights = []
    for l in filtered_layers:
        weights+=l.weights

    # Apply the gradients to the model.
    optimizer.apply_gradients(zip(grads, weights))

: 

Here is the training loop. Each inner iteration runs two batches of size
$2^{12-1}$ simultaneously.

Tensorboard can be used to visualize the training progress. See cell output for
command to start tensorboard.

In [7]:
start_time = time.time()
tf.config.run_functions_eagerly(False)


def check_accuracy(dataset):
    average_loss = 0
    average_accuracy = 0
    for x, y in dataset:
        y = tf.one_hot(tf.cast(y, tf.int32), 2)

        y_pred = vectorize_layer(x)
        # Do not filter when testing.
        for i, l in enumerate(layers):
            y_pred = l(y_pred)
        
        loss = tf.reduce_mean(loss_fn(y, y_pred))

        accuracy = tf.reduce_mean(
            tf.cast(
                tf.equal(tf.argmax(y, axis=1), tf.argmax(y_pred, axis=1)), tf.float32
            )
        )
        average_loss += loss
        average_accuracy += accuracy
    average_loss /= len(dataset)
    average_accuracy /= len(dataset)

    return average_loss, average_accuracy


# Set up tensorboard logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.abspath("") + "/tflogs/sentiment-%s" % stamp
print(f"To start tensorboard, run: tensorboard --logdir ./ --host 0.0.0.0")
print(f"\ttensorboard profiling requires: pip install tensorboard_plugin_profile")
writer = tf.summary.create_file_writer(logdir)

# Initial accuracy
loss, accuracy = check_accuracy(val_data)
tf.print(f"\tvalidation loss: {loss}\taccuracy: {accuracy}")

# Iterate over the batches of the dataset.
for step, (x_batch, y_batch) in enumerate(train_data.take(batch_size)):
    print(
        f"Step: {step} / {len(train_data)}, Time Stamp: {time.time() - start_time}"
    )

    y_batch = tf.one_hot(tf.cast(y_batch, tf.int32), 2)

    if step == 0:
        tf.summary.trace_on(graph=True, profiler=True, profiler_outdir=logdir)

    x_batch = vectorize_layer(x_batch)  # No shape inference, do outside tf.function
    train_step_wrapper(x_batch, y_batch)

    # tf.print("embedding layer slot counter:")
    # tf.print(embedding_layer._last_slot_count, summarize=-1)
    # tf.print("embedding layer max slot counter:")
    # tf.print(tf.reduce_max(embedding_layer._last_slot_count), summarize=-1)

    if step == 0:
        with writer.as_default():
            tf.summary.trace_export(name="sentiment", step=step)

    loss, accuracy = check_accuracy(train_data)
    tf.print(f"\ttrain loss: {loss}\taccuracy: {accuracy}")
    loss, accuracy = check_accuracy(val_data)
    tf.print(f"\tvalidation loss: {loss}\taccuracy: {accuracy}")

    with writer.as_default():
        tf.summary.scalar("loss", loss, step=step)
        tf.summary.scalar("accuracy", accuracy, step=step)


print(f"Total training time: {time.time() - start_time} seconds")

To start tensorboard, run: tensorboard --logdir ./ --host 0.0.0.0
	tensorboard profiling requires: pip install tensorboard_plugin_profile
	validation loss: 0.3467586040496826	accuracy: 0.49951171875


2024-08-21 15:45:05.723558: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-21 15:45:05.756578: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-08-21 15:45:05.756607: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


Step: 0 / 30, Time Stamp: 0.4095344543457031


2024-08-21 16:49:55.676157: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2024-08-21 16:49:55.723325: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-08-21 16:49:55.723922: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: /workspaces/tf-shell/examples/tflogs/sentiment-20240821-154505/plugins/profile/2024_08_21_16_49_55/e64b0b6b3843.xplane.pb


	train loss: 0.3465079069137573	accuracy: 0.5030192136764526


...

Step: 3 / 30, Time Stamp: 11808.985461235046
	train loss: 0.34589430689811707	accuracy: 0.522778332233429


2024-08-21 20:15:12.750700: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


	validation loss: 0.34610289335250854	accuracy: 0.515869140625


2024-08-21 20:15:13.058073: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Step: 4 / 30, Time Stamp: 16207.717448234558
